In [1]:
%gui qt5
import ibapi
from tws_async_qt import TWSClient, iswrapper

In [2]:
# sample application
import datetime
from collections import defaultdict

class TWS(TWSClient):
    def __init__(self):
        TWSClient.__init__(self)
        self._reqIdSeq = 0
        self._histData = defaultdict(list)

    def getReqId(self):
        """
        Get new request ID (integer).
        """
        assert self._reqIdSeq
        newId = self._reqIdSeq
        self._reqIdSeq += 1
        return newId
    
    def popHistData(self, reqId):
        """
        Remove and return the historical data that was downloaded for the given reqId.
        """
        return self._histData.pop(reqId)
    
    @iswrapper
    def nextValidId(self, reqId: int):
        self._reqIdSeq = reqId

    @iswrapper
    def tickPrice(self, reqId: int,
            tickType: ibapi.ticktype.TickType,
            price: float,
            attrib: ibapi.common.TickAttrib):
        print('{} price {}'.format(reqId, price))

    @iswrapper
    def tickSize(self, reqId: int,
            tickType: ibapi.ticktype.TickType,
            size: int):
        print('{} size {}'.format(reqId, size))

    @iswrapper
    def historicalData(self, reqId: int, date: str, open: float, high: float,
            low: float, close: float, volume: int, barCount: int,
            WAP: float, hasGaps: int):
        self._histData[reqId].append((date, open, high, low, close, volume))
        
    @iswrapper
    def historicalDataEnd(self, reqId: int, start: str, end: str):
        print('Historical request {} is finished'.format(reqId))

        
if 'tws' in locals():
    tws.disconnect()
tws = TWS()
tws.connect('maxi', 7496, clientId=1)

ERROR:root:ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR:root:ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR:root:ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR:root:ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR:root:ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR:root:ERROR -1 2104 Market data farm connection is OK:usfarm.us
ERROR:root:ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR:root:ERROR -1 2106 HMDS data farm connection is OK:ilhmds
ERROR:root:ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR:root:ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR:root:ERROR -1 2106 HMDS data farm connection is OK:ushmds


In [3]:
# create a stock contract
contract = ibapi.contract.Contract()
contract.symbol = 'TSLA'
contract.secType = 'STK'
contract.currency = 'USD'
contract.exchange = 'SMART'

In [4]:
# request historical bar data
reqId = tws.getReqId()
tws.reqHistoricalData(reqId, contract,
        endDateTime=datetime.datetime.utcnow().strftime('%Y%m%d %H:%M:%S UTC'),
        durationStr='30 D',
        barSizeSetting='1 hour',
        whatToShow='TRADES',
        useRTH=False,
        formatDate=1,
        chartOptions=None)

Historical request 488 is finished


In [5]:
# process data
import pandas as pd  
data = tws.popHistData(reqId)
df = pd.DataFrame.from_records(data)
df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
df['datetime'] =  pd.to_datetime(df['datetime'])
# df.set_index('datetime', inplace=True)
df

datetime    open    high     low   close  volume
0   2017-03-16 10:00:00  261.10  261.10  261.03  261.10       7
1   2017-03-16 11:00:00  261.24  262.30  261.12  262.05      12
2   2017-03-16 12:00:00  262.00  262.48  261.10  261.10      45
3   2017-03-16 13:00:00  261.50  264.40  261.10  264.00     901
4   2017-03-16 14:00:00  263.74  264.00  259.06  261.37   10356
5   2017-03-16 15:00:00  261.31  261.81  260.10  261.18    8325
6   2017-03-16 16:00:00  261.07  265.75  261.00  264.47   16538
7   2017-03-16 17:00:00  264.56  265.18  263.50  264.53    6016
8   2017-03-16 18:00:00  264.60  265.35  263.44  263.97    4648
9   2017-03-16 19:00:00  264.03  264.14  261.86  263.26    5728
10  2017-03-16 20:00:00  263.29  263.40  261.50  262.08    6896
11  2017-03-16 21:00:00  262.05  262.07  261.60  261.81    1029
12  2017-03-16 22:00:00  261.80  262.07  261.80  261.80      16
13  2017-03-16 23:00:00  261.81  262.10  261.81  262.10      20
14  2017-03-17 00:00:00  262.00  262.00  262.00  262.00       1
15  2017-03-17 11:00:00  262.20  262.20  261.53  262.16      16
16  2017-03-17 12:00:00  262.20  264.05  261.50  262.76      79
17  2017-03-17 13:00:00  263.95  265.00  262.90  263.25     793
18  2017-03-17 14:00:00  263.25  265.09  262.00  263.20   13685
19  2017-03-17 15:00:00  263.19  263.80  262.21  263.43    7391
20  2017-03-17 16:00:00  263.36  264.70  263.00  264.20    5618
21  2017-03-17 17:00:00  264.13  265.33  263.92  264.80    5217
22  2017-03-17 18:00:00  264.73  264.96  263.15  263.37    4845
23  2017-03-17 19:00:00  263.38  263.56  261.46  262.55    6857
24  2017-03-17 20:00:00  262.62  262.68  261.20  261.29    7208
25  2017-03-17 21:00:00  261.50  261.50  261.09  261.25    1826
26  2017-03-17 22:00:00  261.15  261.50  261.09  261.27     125
27  2017-03-17 23:00:00  261.30  261.30  261.08  261.08      13
28  2017-03-18 00:00:00  261.08  261.38  261.08  261.15      29
29  2017-03-20 10:00:00  261.25  261.25  260.18  260.18      23
..                  ...     ...     ...     ...     ...     ...
420 2017-04-25 17:00:00  308.87  311.46  308.85  310.72    8313
421 2017-04-25 18:00:00  310.81  313.17  310.72  311.89    8488
422 2017-04-25 19:00:00  311.83  313.68  311.32  312.88    6705
423 2017-04-25 20:00:00  312.84  313.49  311.66  311.95    4927
424 2017-04-25 21:00:00  311.99  313.98  311.98  313.80    9704
425 2017-04-25 22:00:00  313.73  313.85  313.51  313.51    1208
426 2017-04-25 23:00:00  313.70  313.75  313.51  313.51      20
427 2017-04-26 00:00:00  313.45  313.60  313.45  313.55       3
428 2017-04-26 01:00:00  313.60  313.75  313.60  313.75       5
429 2017-04-26 10:00:00  313.14  313.99  313.14  313.99       2
430 2017-04-26 11:00:00  313.59  313.59  313.59  313.59      10
431 2017-04-26 12:00:00  313.30  313.35  313.00  313.12      27
432 2017-04-26 13:00:00  312.80  313.10  311.52  311.70      28
433 2017-04-26 14:00:00  312.35  313.00  311.17  312.36     250
434 2017-04-26 15:00:00  312.36  314.50  311.26  312.00    8833
435 2017-04-26 16:00:00  311.96  312.83  310.25  312.68    7596
436 2017-04-26 17:00:00  312.75  312.89  311.01  311.30    4057
437 2017-04-26 18:00:00  311.43  312.00  310.78  311.18    2462
438 2017-04-26 19:00:00  311.14  312.04  310.81  310.94    2787
439 2017-04-26 20:00:00  311.02  311.02  309.26  309.45    3839
440 2017-04-26 21:00:00  309.47  310.60  309.00  310.17    5785
441 2017-04-26 22:00:00  310.17  310.17  309.80  309.80     430
442 2017-04-26 23:00:00  310.17  310.17  309.95  309.95       2
443 2017-04-27 00:00:00  310.00  310.20  309.75  310.20      22
444 2017-04-27 01:00:00  310.20  310.20  310.01  310.01       2
445 2017-04-27 10:00:00  309.90  309.90  309.90  309.90       1
446 2017-04-27 11:00:00  309.90  309.90  309.90  309.90       0
447 2017-04-27 12:00:00  309.65  309.65  309.65  309.65       1
448 2017-04-27 13:00:00  309.90  310.10  309.90  310.10      17
449 2017-04-27 14:00:00  310.59  310.59  310.47  310.59      19

[450 rows x 6 columns]

In [6]:
# plot data
%matplotlib inline
import seaborn as sns
df.plot(y='close', figsize=(12,8))

In [7]:
# subscribe to realtime tick data
reqId = tws.getReqId()
tws.reqMktData(reqId, contract, genericTickList='', snapshot=False,
        regulatorySnapshot=False, mktDataOptions=[])

489 price 310.2
489 size 3
489 price 310.59
489 size 2
489 price 310.59
489 size 1
489 size 3
489 size 2
489 size 1
489 size 42
489 price 310.17


In [8]:
# cancel realtime ticks
tws.cancelMktData(reqId)

In [9]:
# asyncio integration
import asyncio
import quamash

async def coro(seconds):
    print('starting coroutine...')
    await asyncio.sleep(seconds)
    print('coroutine finished')

loop = quamash.QEventLoop()
asyncio.set_event_loop(loop)

task = loop.create_task(coro(seconds=10))
loop.run_forever()

-1

starting coroutine...
coroutine finished
